In [1]:
!pip install overrides
!pip install allennlp

  Stored in directory: /root/.cache/pip/wheels/8d/52/86/e5a83b1797e7d263b458d2334edd2704c78508b3eea9323718
Successfully built overrides
    100% |████████████████████████████████| 5.6MB 7.4MB/s 
    100% |████████████████████████████████| 51kB 19.5MB/s 
    100% |████████████████████████████████| 552kB 24.5MB/s 
    100% |████████████████████████████████| 133kB 30.4MB/s 
    100% |████████████████████████████████| 512kB 23.7MB/s 
    100% |████████████████████████████████| 4.5MB 7.6MB/s 
    100% |████████████████████████████████| 12.6MB 3.9MB/s 
    100% |████████████████████████████████| 122kB 32.2MB/s 
    100% |████████████████████████████████| 245kB 26.6MB/s 
    100% |████████████████████████████████| 51kB 21.1MB/s 
    100% |████████████████████████████████| 1.5MB 16.1MB/s 
    100% |████████████████████████████████| 143kB 32.0MB/s 
    100% |████████████████████████████████| 2.3MB 13.6MB/s 
    100% |████████████████████████████████| 61kB 24.4MB/s 
    100% |███████████████████

In [13]:
from typing import Iterator, List, Dict
import logging, csv, re
import torch
from torchsummary import summary
import torchtext
import torch.optim as optim
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm
from overrides import overrides
from allennlp.data import Instance
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.tokenizers import Tokenizer, WordTokenizer
from allennlp.data.tokenizers.word_splitter import JustSpacesWordSplitter
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.fields import LabelField, TextField, Field, SequenceLabelField
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.data.iterators import BucketIterator

from allennlp.training.trainer import Trainer

from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.token_indexers.elmo_indexer import ELMoTokenCharactersIndexer
from allennlp.modules.token_embedders import ElmoTokenEmbedder

from allennlp.nn.util import get_text_field_mask
from allennlp.training.metrics import CategoricalAccuracy, F1Measure
from allennlp.models import Model

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

logger = logging.getLogger(__name__) # pylint: disable=invalid-name
EMBEDDING_DIM = 128
HIDDEN_DIM = 60 #128
MAX_LEN = 70
dropout = 0.25
lstm_layers = 2

import io, random, os
import tarfile
import urllib.request
import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

tqdm.pandas(desc='Progress')

# cross validation and metrics
from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split

SEED = 1033

def seed_everything(seed=1033):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

data_dir = ""

cuda:0


In [0]:
text_list = []

def binaryStreamToList(source,dir_name):
    res = []
    
    source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
    tar = tarfile.open(fileobj=urllib.request.urlopen(source), mode="r|gz")
    for member in tar:
        if member.name.endswith(".txt") and member.name.startswith(dir_name):
            text_from_file = ""
            content = tar.extractfile(member).read()
            with io.BytesIO(content) as f:
                for line in f:
                    text_from_file += str(line).strip()
                res.append(text_from_file)
    return res

def collect_df(source,dir_name,target):
    text_list = binaryStreamToList(source,dir_name)
    df = pd.DataFrame({'review_text': text_list,
                  'target': target})
    return df  
  
source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

In [0]:
test_pos_tar = 'aclImdb/test/pos/'
test_neg_tar = 'aclImdb/test/neg/'
train_pos_tar = 'aclImdb/train/pos/'
train_neg_tar = 'aclImdb/train/neg/'

pos_train_df = collect_df(source,train_pos_tar,1)
neg_train_df = collect_df(source,train_neg_tar,0)
pos_test_df = collect_df(source,test_pos_tar,1)
neg_test_df = collect_df(source,test_neg_tar,0)

all_df = pd.concat([pos_train_df ,neg_train_df,pos_test_df ,neg_test_df]).sample(frac=1)

In [7]:
train_df, test_df = train_test_split(all_df, test_size=0.2)

#check for duplicates
duplicateRowsDF = all_df[all_df.duplicated()]
print(len(train_df[train_df['target']==1]))
print(len(train_df[train_df['target']==0]))
print(len(test_df[test_df['target']==1]))
print(len(test_df[test_df['target']==0]))

train_df.to_csv("train.csv")
test_df.to_csv("test.csv")

20044
19956
4956
5044


In [0]:
def known_contractions(embed):
    known = []
    for contract in contraction_mapping:
        if contract in embed:
            known.append(contract)
    return known
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text
def correct_spelling(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x
def unknown_punct(embed, punct):
    unknown = ''
    for p in punct:
        if p not in embed:
            unknown += p
            unknown += ' '
    return unknown

def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  # Other special characters that I have to deal with in last
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text
def add_lower(embedding, vocab):
    count = 0
    for word in vocab:
        if word in embedding and word.lower() not in embedding:  
            embedding[word.lower()] = embedding[word]
            count += 1
    print(f"Added {count} words to embedding")    

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)
  
def preprocess_text(text):
    return replace_typical_misspell(clean_numbers(clean_text(text.strip().lower())))

In [0]:
#@DatasetReader.register("imdb_sentiment")
class ImdbSentimentDatasetReader(DatasetReader):
    def __init__(self,
                 lazy: bool = False,
                 tokenizer: Tokenizer = None,
                 token_indexers: Dict[str, TokenIndexer] = None) -> None:
        super().__init__(lazy)
        self._tokenizer = tokenizer or WordTokenizer(JustSpacesWordSplitter())
        self._token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}

    @overrides
    def _read(self, file_path):
        logger.info("Reading instances from lines in file at: %s", file_path)
        with open(cached_path(file_path), "r") as data_file:
            csv_in = csv.reader(data_file, delimiter=',')
            next(csv_in)
            for row in csv_in:
                if len(row) == 3:
                    review_text = preprocess_text(row[1])
                    labels_list.append(row[3])
                    yield self.text_to_instance(text=review_text, label=row[2])
    @overrides
    def text_to_instance(self,  # type: ignore
                         text: str,
                         label: str = None) -> Instance:
        # pylint: disable=arguments-differ
        fields: Dict[str, Field] = {}
        tokenized_text = self._tokenizer.tokenize(text)
        #torch.tensor(tokenized_text, dtype=torch.long).cuda()
        fields["tokens"] = TextField(tokenized_text[:MAX_LEN], self._token_indexers)
        if label is not None:
            fields['label'] = LabelField(label)
        return Instance(fields)

In [20]:
labels_list = []

elmo_token_indexer = ELMoTokenCharactersIndexer()
reader = ImdbSentimentDatasetReader(
    token_indexers={'tokens': elmo_token_indexer})

train_dataset = reader.read(data_dir+"train.csv")
dev_dataset = reader.read(data_dir+"test.csv")


0it [00:00, ?it/s]
74it [00:00, 731.27it/s]
158it [00:00, 759.63it/s]
231it [00:00, 747.91it/s]
310it [00:00, 759.87it/s]
394it [00:00, 779.39it/s]
485it [00:00, 813.77it/s]
569it [00:00, 818.55it/s]
655it [00:00, 827.59it/s]
738it [00:00, 826.55it/s]
40000it [00:52, 755.70it/s]
10000it [00:12, 791.00it/s]


In [21]:

# Initialize the ELMo-based token embedder using a pre-trained file.
# This takes a while if you run this script for the first time

# Original
# options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
# weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

# Medium
# options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x2048_256_2048cnn_1xhighway/elmo_2x2048_256_2048cnn_1xhighway_options.json"
# weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x2048_256_2048cnn_1xhighway/elmo_2x2048_256_2048cnn_1xhighway_weights.hdf5"

# Use the 'Small' pre-trained model
options_file = ('https://s3-us-west-2.amazonaws.com/allennlp/models/elmo'
                '/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_options.json')
weight_file = ('https://s3-us-west-2.amazonaws.com/allennlp/models/elmo'
               '/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5')

elmo_embedder = ElmoTokenEmbedder(options_file, weight_file,2, dropout=0.1)

vocab = Vocabulary.from_instances(train_dataset + dev_dataset,
                                  min_count={'tokens': 3})

# Pass in the ElmoTokenEmbedder instance instead
word_embeddings = BasicTextFieldEmbedder({"tokens": elmo_embedder.to(device)})

100%|██████████| 50000/50000 [00:00<00:00, 55727.30it/s]


In [0]:

class LstmTwoClassifier(Model):
    def __init__(self,
                 word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 vocab: Vocabulary) -> None:
        super().__init__(vocab)
        # We need the embeddings to convert word IDs to their vector representations
        self.word_embeddings = word_embeddings


        self.encoder = encoder


        self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                          out_features=2)
        self.accuracy = CategoricalAccuracy()
        #self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.out_act = torch.nn.Sigmoid()
        

        #self.loss_function = torch.nn.BCELoss()   #BCEWithLogitsLoss()   #CrossEntropyLoss()
        self.loss = torch.nn.CrossEntropyLoss()  #torch.nn.BCEWithLogitsLoss()
        

    def forward(self,
                tokens: Dict[str, torch.tensor],
                label: torch.tensor = None) -> torch.tensor:

        mask = get_text_field_mask(tokens)

        # Forward pass
        embeddings = self.word_embeddings(tokens)
        encoder_out = self.encoder(embeddings, mask)
        linear_out = self.hidden2tag(encoder_out)
        
        logits = self.out_act(linear_out)
        
        output = {"logits": logits}
        if label is not None:
            #y = torch.tensor(label.reshape(-1, 1), dtype=torch.float).cuda()
            self.accuracy(logits, label)
            #output["loss"] = self.loss_function(logits, label)
            output["loss"] = self.loss(logits, label)
            
            
        return output

In [0]:

elmo_embedding_dim = 256
lstm = PytorchSeq2VecWrapper(
    torch.nn.LSTM(elmo_embedding_dim, HIDDEN_DIM, bidirectional=True,num_layers=lstm_layers, 
                            dropout = dropout, batch_first=True))

#model = LstmBinaryClassifier(word_embeddings, lstm, vocab).cuda()

model = LstmTwoClassifier(word_embeddings, lstm, vocab).cuda()

optimizer = optim.Adam(model.parameters())

loss_function = torch.nn.BCEWithLogitsLoss()

batch_size = 128
iterator = BucketIterator(batch_size=batch_size, sorting_keys=[("tokens", "num_tokens")])

#train_iter = torchtext.data.BucketIterator(dataset=train_dataset,batch_size=batch_size, sort_key=[("tokens", "num_tokens")])
#val_iter = torchtext.data.BucketIterator(dataset=dev_dataset,batch_size=batch_size, sort_key=[("tokens", "num_tokens")])


iterator.index_with(vocab)
#train_iterator.index_with(vocab)
#val_iterator.index_with(vocab)


def save(m, info):
    torch.save(info, data_dir+'best_model.info')
    torch.save(m, data_dir+'best_model.m')
    
def load():
    m = torch.load(data_dir+'best_model.m')
    info = torch.load(data_dir+'best_model.info')
    return m, info

trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=dev_dataset,
                  patience=2,
                  cuda_device=0,
                  num_epochs=5)

#summary(model, input_size=(1, 28, 28))
#training(model=model, epoch=1, eval_every=500,
#         loss_func=loss_function, optimizer=optimizer, train_iter=train_iter,
#        val_iter=val_iter, warmup_epoch=0, early_stop=2)

In [24]:
trainer.train()

loss: 0.6354 ||: 100%|██████████| 313/313 [04:21<00:00,  1.38it/s]
loss: 0.6051 ||: 100%|██████████| 79/79 [01:00<00:00,  1.90it/s]
loss: 0.5848 ||: 100%|██████████| 313/313 [03:51<00:00,  1.36it/s]
loss: 0.5767 ||: 100%|██████████| 79/79 [00:51<00:00,  1.90it/s]
loss: 0.5544 ||: 100%|██████████| 313/313 [04:15<00:00,  1.35it/s]
loss: 0.5471 ||: 100%|██████████| 79/79 [00:50<00:00,  1.91it/s]
loss: 0.5311 ||: 100%|██████████| 313/313 [03:51<00:00,  1.34it/s]
loss: 0.5293 ||: 100%|██████████| 79/79 [00:51<00:00,  1.91it/s]
loss: 0.5225 ||: 100%|██████████| 313/313 [03:56<00:00,  1.36it/s]
loss: 0.5219 ||: 100%|██████████| 79/79 [00:51<00:00,  1.90it/s]


{'best_epoch': 4,
 'best_validation_loss': 0.5218760948392409,
 'epoch': 4,
 'peak_cpu_memory_MB': 10214.496,
 'peak_gpu_0_memory_MB': 3646,
 'training_cpu_memory_MB': 10214.496,
 'training_duration': '00:24:44',
 'training_epochs': 4,
 'training_gpu_0_memory_MB': 3646,
 'training_loss': 0.5224993031817122,
 'training_start_epoch': 0,
 'validation_loss': 0.5218760948392409}

In [26]:
model_file_path = data_dir+"model_allen_imdb_twoclass.th"
vocab_file_path = data_dir+"vocabulary_allennlp_imdb_twoclass"

with open(model_file_path, 'wb') as f:
    torch.save(model.state_dict(), f)

vocab.save_to_files(vocab_file_path)



In [27]:
!ls

'glove.6B.300d (1).txt'         test.csv			    x_train.npy
 glove.6B.300d.txt	        train.csv			    y_test.npy
 glove.6B.zip		        vocabulary_allennlp_imdb_twoclass   y_train.npy
 model_allen_imdb_twoclass.th   word_index.npy
 sample_data		        x_test.npy


In [30]:
vocab

Vocabulary with namespaces:  labels, Size: 2 || Non Padded Namespaces: {'*tags', '*labels'}

In [0]:


from allennlp.common import JsonDict
from allennlp.data import DatasetReader, Instance
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.predictors import Predictor
from overrides import overrides
from typing import List

#@Predictor.register("sentence_classifier_predictor")
class SentenceClassifierPredictor(Predictor):
    def __init__(self, model: Model, dataset_reader: DatasetReader) -> None:
        super().__init__(model, dataset_reader)
        self._tokenizer = SpacyWordSplitter(language='en_core_web_sm', pos_tags=True)

    def predict(self, sentence: str) -> JsonDict:
        return self.predict_json({"sentence" : sentence})

    @overrides
    def _json_to_instance(self, json_dict: JsonDict) -> Instance:
        sentence = json_dict["sentence"]
        tokens = self._tokenizer.split_words(sentence)
        #return self._dataset_reader.text_to_instance([str(t) for t in tokens])
        return self._dataset_reader.text_to_instance(sentence)

In [33]:
tests = ['this movie sucks','I love this movie','wow this is the worst film ever','I hated it, really. Do not watch this','it is pretty good, I would see it again']

for test in tests:
  predictor = SentenceClassifierPredictor(model, dataset_reader=reader)
  logits = predictor.predict(test)['logits']
  label_id = np.argmax(logits)

  print(model.vocab.get_token_from_index(label_id, 'labels'))
  
  print(logits)
  

0
[0.9339547753334045, 0.06673074513673782]
1
[0.010839399881660938, 0.9916225075721741]
0
[0.9991610050201416, 0.0006861151778139174]
0
[0.6092225909233093, 0.40380367636680603]
1
[0.06797058880329132, 0.9387489557266235]
